<a href="https://colab.research.google.com/github/merriekay/CS167Code/blob/main/Notebook2_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 2
## CS167: Machine Learning
Meredith Moore

### Instructions:
For this notebook, you're going to continue working with the Vehicles data that you worked with in Notebook 1. You're going to write a function that can make k-Nearest-Neighbor predictions for the combined fuel efficiency ("comb08" column) based on three predictor columns: the year, the number of engine cylinders, and the engine displacement in liters ("year", "cylinders", and "displ" columns).

## Part 1
1. Work with the right subset: You're not going to work with the whole data set, just the cars with "Regular" listed in the "FuelType" column (in Notebook 1, you looked the regular fuel cars since 2020, but in this one, you will use all years in the data set). So, to start, make a subset of the original data.

In [5]:
#Part 1:
import pandas  
from google.colab import drive
drive.mount('/content/drive')
vehicles = pandas.read_csv('/content/drive/MyDrive/CS167_S21/CS167_datasets/vehicles.csv')
pandas.set_option('display.max_columns', 100)

#make the subset.
#gas_vehicles = vehicles[vehicles['fuelType']=="Regular"][['year', 'cylinders', 'displ', 'comb08']] #one-liner

gas_vehicles = vehicles[vehicles['fuelType']=="Regular"]
gas_vehicles = gas_vehicles[['year', 'cylinders', 'displ', 'comb08']]
gas_vehicles

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,cylinders,displ,comb08
0,1985,4.0,2.0,21
1,1985,12.0,4.9,11
2,1985,4.0,2.2,27
3,1985,8.0,5.2,11
5,1993,4.0,1.8,22
...,...,...,...,...
42097,1993,4.0,1.9,25
42098,1993,4.0,2.2,22
42099,1993,4.0,2.2,23
42100,1993,4.0,2.2,21


## Part 2:
1. Check for null values in the target and predictor columns. 
2. If you have a null target value, you will need to throw that example out.
3. If you have a null predictor value, you can either fill them in with something (like the mean/median) or you can drop those rows from the data set.
4. Describe in a markup cell what you decided to do with the null data.

In [6]:
#Display columns that have missing data.
print(gas_vehicles.columns[gas_vehicles.isna().any()])


Index(['cylinders', 'displ'], dtype='object')


From the cell above, we can see that the 'cylinders' and 'displ' columns have null values, while 'year' and 'comb08', our target function, does not have any null values. 

In [7]:
#copy the data to try and see what dropping all NaN values will do:
drop_test = gas_vehicles.copy()
drop_test.dropna(inplace=True)
drop_test.shape

(26860, 4)

### Handling Missing Data
To handle the missing data, I first determined which columns had missing data and found that the 'cylinders' column and 'displ' columns each had missing data. I then tested how many data points would be lost if I used dropna() indiscriminantly, and it only dropped 3 rows. Since this only loses 3 rows, I think that using dropna() is a reasonable way to deal with the missing data.


In [8]:
gas_vehicles.dropna(inplace=True)
gas_vehicles.head()

,year,cylinders,displ,comb08
0,1985,4.0,2.0,21
1,1985,12.0,4.9,11
2,1985,4.0,2.2,27
3,1985,8.0,5.2,11
5,1993,4.0,1.8,22


# Part 3:
3. Write up a k-nearest-neighbors function like the one you made for the iris data set in class. It should be able to make mpg ("comb08") predictions for new cars based on the year, cylinders, and displacement. You should also be able to specify what you want to use as k.

In [9]:
import numpy 
def knn(car, data, k):
    
    #calculate distances
    data['distance_to_new'] = numpy.sqrt((car['year'] - data['year'])**2
                                         +(car['cylinders']-data['cylinders'])**2
                                         +(car['displ']-data['displ'])**2)
    
    #sort
    sorted_data = data.sort_values(['distance_to_new'])
    print(sorted_data.iloc[0:k])
    
    #return the mode of the top k
    return sorted_data.iloc[0:k]['comb08'].mean()

# Part 4:
4. Demonstrate that your function works by making up some new values for hypothetical cars and using your function to display the predicted mpg ("comb08") for that car.

In [10]:
#Non-Normalized Calculation
car = {}
car['year'] = 1995
car['cylinders'] = 8
car['displ'] = 4

not_normal_pred = knn(car, gas_vehicles, 5)
not_normal_pred

      year  cylinders  displ  comb08  distance_to_new
2246  1995        8.0    4.3      19              0.3
2281  1995        8.0    4.6      16              0.6
2286  1995        8.0    4.6      18              0.6
2285  1995        8.0    4.6      18              0.6
2284  1995        8.0    4.6      18              0.6


17.8

# Part 5:
5. Make a copy of the data and normalize the training data using Z-score. Predict the mpg of a particular car using your k-nearest-neighbors function with both the normalized and non-normalized training data. Compare your results. Use a markup cell to describe and explain the differences in a few sentences.

In [11]:
z_gas_vehicles = gas_vehicles.copy()

#calculate mean and std for 'year', 'cylinder', and 'displ'
year_mean = z_gas_vehicles['year'].mean()
year_std= z_gas_vehicles['year'].std()
cylinders_mean = z_gas_vehicles['cylinders'].mean()
cylinders_std= z_gas_vehicles['cylinders'].std()
displ_mean = z_gas_vehicles['displ'].mean()
displ_std= z_gas_vehicles['displ'].std()

#calculate z-scores for columns
z_gas_vehicles['year'] = (z_gas_vehicles['year'] - year_mean)/year_std
z_gas_vehicles['cylinders'] = (z_gas_vehicles['cylinders'] - cylinders_mean)/cylinders_std
z_gas_vehicles['displ'] = (z_gas_vehicles['displ'] - displ_mean)/displ_std

#calculate the z-score of the new test example:
car = {}
car['year'] = (1995 - year_mean)/year_std
car['cylinders'] = (8- cylinders_mean)/cylinders_mean
car['displ'] = (4- displ_mean)/displ_mean

normal_pred = knn(car, z_gas_vehicles, 5)
normal_pred

          year  cylinders     displ  comb08  distance_to_new
2268 -0.355179   0.415896  0.301551      19         0.076150
2251 -0.355179   0.415896  0.301551      19         0.076150
2252 -0.355179   0.415896  0.301551      19         0.076150
2265 -0.355179   0.415896  0.301551      19         0.076150
2587 -0.355179   0.415896  0.222964      16         0.095656


18.4